In [2]:
import pandas as pd
from src.train import *
from src.model import CNNLSTMModel
from src.dataset import *
from src.preprocess import *
from src.visualize import *
import torch

In [ ]:
# load dataset
train_df = pd.read_csv('dataset/train.csv')
val_df = pd.read_csv('dataset/val.csv')
test_df = pd.read_csv('dataset/test.csv')

In [6]:
# load model
model = CNNLSTMModel(num_features=train_df.shape[1]-3, lstm_layers=2)
model.load_state_dict(torch.load('model/best_model.pth'))
model.eval()

CNNLSTMModel(
  (cnn): Sequential(
    (0): Conv1d(21, 32, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
  )
  (lstm): LSTM(32, 64, num_layers=2, batch_first=True, dropout=0.3)
  (classifier): Sequential(
    (0): Linear(in_features=64, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [12]:
all_col = train_df.columns
label_col = 'label'
device_col = 'device_name'
time_col = 'event_local_time'
feature_col = list(all_col.difference([label_col, device_col, time_col]))

In [ ]:

window_size = 2*24*7 # 30 minutes*2*24*7 = 7 days
normal_stride = 2*6 # 30 minutes*2*6 = 6 hours

train_ds = InverterTimeSeriesDataset.from_dataframe(train_df, feature_col, window_size=window_size, stride=normal_stride)
val_ds   = InverterTimeSeriesDataset.from_dataframe(val_df,   feature_col, window_size=window_size, stride=normal_stride)
test_ds  = InverterTimeSeriesDataset.from_dataframe(test_df,  feature_col, window_size=window_size, stride=normal_stride)

Processing devices: 100%|██████████| 16/16 [00:00<00:00, 122.93it/s]


In [15]:
from torch.utils.data import DataLoader
train_loader = DataLoader(train_ds, batch_size=1024, shuffle=True, num_workers=8, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=1024, shuffle=False, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=1024, shuffle=False, num_workers=8, pin_memory=True)

In [17]:
train_trues, train_outputs, train_loss= test_loop(model, train_loader)
val_trues, val_outputs, val_loss = test_loop(model, val_loader)
test_trues, test_outputs, test_loss = test_loop(model, test_loader)

Testing: 100%|██████████| 11/11 [00:06<00:00,  1.73it/s]
